In [1]:
## for the follwing code

#         # Embedding block.
#         x = self.emb(data.atomic_numbers.long(), rbf, i, j)
#         P = self.output_blocks[0](x, rbf, i, num_nodes=pos.size(0))
#         print(x.shape, P.shape)
        
#         ## change
#         def capture_embeddings(module, input, output):
#             self.embs.append(output.detach())  # Store the output embeddings
            
#         ## change
#         hook_handle = self.interaction_blocks[-1].register_forward_hook(capture_embeddings)
            
#         # Interaction blocks.
#         for interaction_block, output_block in zip(
#             self.interaction_blocks, self.output_blocks[1:]
#         ):
#             x = interaction_block(x, rbf, sbf, idx_kj, idx_ji)
#             P += output_block(x, rbf, i, num_nodes=pos.size(0))
#             print(x.shape, P.shape)

## getting following output

# Atoms(symbols='H5Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# Atoms(symbols='CH9Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# -1.2942085266113281

In [2]:
# https://nbviewer.org/gist/mhoffman/c418acb6b3f928eb4ef71f4001d120d9

In [3]:
import requests
import pprint
import sys
import string
import json
import io
import copy

import ase.io
import ase.calculators.singlepoint

GRAPHQL = 'http://api.catalysis-hub.org/graphql'

def fetch(query):
    return requests.get(
        GRAPHQL, {'query': query}
    ).json()['data']

In [4]:
def reactions_from_dataset(pub_id, page_size=10):
    reactions = []
    has_next_page = True
    start_cursor = ''
    page = 0
    while has_next_page:
        data = fetch("""{{
      reactions(pubId: "{pub_id}", first: {page_size}, after: "{start_cursor}") {{
        totalCount
        pageInfo {{
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor 
        }}  
        edges {{
          node {{
            Equation
            reactants
            products
            reactionEnergy
            reactionSystems {{
              name
              systems {{
                energy
                InputFile(format: "json")
              }}
            }}  
          }}  
        }}  
      }}    
    }}""".format(start_cursor=start_cursor,
                 page_size=page_size,
                 pub_id=pub_id,
                ))
        has_next_page = data['reactions']['pageInfo']['hasNextPage']
        start_cursor = data['reactions']['pageInfo']['endCursor']
        page += 1
        print(has_next_page, start_cursor, page_size * page, data['reactions']['totalCount'])
        reactions.extend(map(lambda x: x['node'], data['reactions']['edges']))

    return reactions

# raw_reactions = reactions_from_dataset("MamunHighT2019") # this will get all 88587 reactions
raw_reactions = reactions_from_dataset("FesterEdge2017")

True YXJyYXljb25uZWN0aW9uOjk= 10 16
False YXJyYXljb25uZWN0aW9uOjE1 20 16


In [5]:
def aseify_reactions(reactions):
    for i, reaction in enumerate(reactions):
        for j, _ in enumerate(reactions[i]['reactionSystems']):
            with io.StringIO() as tmp_file:
                system = reactions[i]['reactionSystems'][j].pop('systems')
                tmp_file.write(system.pop('InputFile'))
                tmp_file.seek(0)
                atoms = ase.io.read(tmp_file, format='json')
            calculator = ase.calculators.singlepoint.SinglePointCalculator(
                atoms,
                energy=system.pop('energy')
            )
            atoms.set_calculator(calculator)
            #print(atoms.get_potential_energy())
            reactions[i]['reactionSystems'][j]['atoms'] = atoms
        # flatten list further into {name: atoms, ...} dictionary
        reactions[i]['reactionSystems'] = {x['name']: x['atoms']
                                          for x in reactions[i]['reactionSystems']}
        
reactions = copy.deepcopy(raw_reactions)
aseify_reactions(reactions)

In [6]:
reactions[5]

{'Equation': '2.0H2O(g) - 1.5H2(g) + * -> HO2*',
 'reactants': '{"star": 1, "H2gas": -1.5, "H2Ogas": 2.0}',
 'products': '{"HO2star": 1}',
 'reactionEnergy': 1.8164483250000103,
 'reactionSystems': {'HO2star': Atoms(symbols='H6Au48Co8O19', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=SinglePointCalculator(...)),
  'H2Ogas': Atoms(symbols='H2O', pbc=True, cell=[14.0, 16.526478, 16.596309], calculator=SinglePointCalculator(...)),
  'H2gas': Atoms(symbols='H2', pbc=True, cell=[14.0, 15.0, 16.737166], calculator=SinglePointCalculator(...)),
  'star': Atoms(symbols='H5Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=SinglePointCalculator(...))}}

In [7]:
reactions[5]['reactionSystems']['star']  # this is my slab

Atoms(symbols='H5Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=SinglePointCalculator(...))

In [8]:
# this is just a copy for feature generator

In [10]:
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator

m = MPRester('Yct0KDbJbqMLWluZEovkwrLXh2VRHXbc')

config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"

# Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

/home/mchowdh5/.local/lib/python3.10/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/opt/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [11]:
calc

In [12]:
def get_energy(ase_slab, ads_symbol):
    
    adslab = ase_slab.copy()
    adslab.calc = calc
    print(adslab)
    
    slab_energy = adslab.get_potential_energy()
    
    # add the molecule
    adsorbate = molecule(ads_symbol)
    add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
    print(adslab)
    
    ads_energy = adslab.get_potential_energy()
    
    # because this is a fingerprint we do not need to reference to gas molecules 
    # because this is the same constant across surfaces
    
    return ads_energy - slab_energy
    

get_energy( reactions[5]['reactionSystems']['star'], 'CH4' )

Atoms(symbols='H5Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))
Atoms(symbols='CH9Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))


-1.2942085266113281

In [13]:
embeddings = calc.trainer.model.module.embs

In [14]:
a = embeddings[0]

In [15]:
a.shape

torch.Size([3533, 256])

In [16]:
a[0, :5]

tensor([-0.5188,  0.0327, -0.5164, -0.8912,  2.3772])

tensor([-0.5188,  0.0327, -0.5164, -0.8912,  2.3772])

In [17]:
a[1, :5]

tensor([-0.5305,  0.0348, -0.5109, -0.8876,  2.3865])

tensor([-0.5305,  0.0348, -0.5109, -0.8876,  2.3865])

In [19]:
import torch
b = torch.mean(a, dim=0)


In [22]:
b.shape

torch.Size([256])